## 10th Circuit Court Opinion Data Bibliography

#### Overview

The data is  from [Court Listener](https://www.courtlistener.com), a Free Law Project...project. [Free Law Project](https://free.law) is a data/software oriented nonprofit seeking to "make the legal ecosystem more equitable and competitive." The [Court Listener API](https://www.courtlistener.com/api/) allows for bulk dowloads of court proceedings including arguments and opinions from each Circuit Court and the Supreme Court. I found the Court Listener API this [this tutorial](https://github.com/idc9/word_embed_tutorial) which creates a word embedding with Supreme Court opinions.

Note: put the opinions from [here](https://www.courtlistener.com/api/bulk-data/opinions/ca10.tar.gz) in the same directory and name directory 'ca10'

### Contents

In [2]:
import glob
import os
json_files = glob.glob('/Users/keira.richards/E_DHML/embed/ca10/*' + "*.json")
len(json_files)

56194

The data set is 56194 JSON files of court opinions from the 10th circuit court. Each file is one opinion on one case. The cases are chornological, but the numbered files arent, alluding to possible missing cases. it begins with 95.json from march 13, 2010 and ends with 6216474.json from Febuary 8, 2022. The data does not go as far back as the Supreme Court opinions available on Court Listiner, but will lend itself well to analyzing change over the last decade. I'm going to read in the first file to look at the contents qualitatively 

In [5]:
op95= open('ca10/95.json').read()
op95[:3000]

'{\n  "resource_uri": "https://www.courtlistener.com/api/rest/v3/opinions/95/",\n  "id": 95,\n  "absolute_url": "/opinion/95/united-states-v-frownfelter/",\n  "cluster": "https://www.courtlistener.com/api/rest/v3/clusters/95/",\n  "author": null,\n  "joined_by": [],\n  "author_str": "",\n  "per_curiam": false,\n  "joined_by_str": "",\n  "date_created": "2010-03-13T23:59:33Z",\n  "date_modified": "2020-03-02T19:53:50.186931Z",\n  "type": "010combined",\n  "sha1": "ee78eebf5c1aaa2cfa3f382c1b364e6573f0067c",\n  "page_count": 7,\n  "download_url": "http://www.ca10.uscourts.gov/opinions/10/10-4016.pdf",\n  "local_path": "pdf/2010/03/11/United_States_vs._Frownfelter_1.pdf",\n  "plain_text": "                                                                        FILED\\n                                                             United States Court of Appeals\\n                                                                     Tenth Circuit\\n\\n                                           

Ew, Lets see if i can make it more readable

In [6]:
import re
op95_readable = re.split(r"\n|\\n", op95)
op95_readable[:50]

['{',
 '  "resource_uri": "https://www.courtlistener.com/api/rest/v3/opinions/95/",',
 '  "id": 95,',
 '  "absolute_url": "/opinion/95/united-states-v-frownfelter/",',
 '  "cluster": "https://www.courtlistener.com/api/rest/v3/clusters/95/",',
 '  "author": null,',
 '  "joined_by": [],',
 '  "author_str": "",',
 '  "per_curiam": false,',
 '  "joined_by_str": "",',
 '  "date_created": "2010-03-13T23:59:33Z",',
 '  "date_modified": "2020-03-02T19:53:50.186931Z",',
 '  "type": "010combined",',
 '  "sha1": "ee78eebf5c1aaa2cfa3f382c1b364e6573f0067c",',
 '  "page_count": 7,',
 '  "download_url": "http://www.ca10.uscourts.gov/opinions/10/10-4016.pdf",',
 '  "local_path": "pdf/2010/03/11/United_States_vs._Frownfelter_1.pdf",',
 '  "plain_text": "                                                                        FILED',
 '                                                             United States Court of Appeals',
 '                                                                     Tenth 

In [7]:
#uncomment the following to read the whole thing
#print(op95_readable)

Good enough, and look at all that helpful metadata at the top. Reading it as a human, the files contain a breif summary of the case, the decision of the court and the rational behind the decision. The author meta for all the files ones ive looked at are blank, but in the files they all seem to be written by respective "clerks of the court". That makes sense given their job is record keeping, but its hard to tell if this was what the judge said in court or was dictated later. I notice in the media it says '"per curiam" = flase' meaning its not a judges decison. I think i will try to figure out if i can narrow down the opinions to just per curiam opinions using the meta data

In [56]:
ca10='ca10/'
for file_name in os.listdir(ca10):
    file=os.path.join(ca10,file_name)
    txt=open(file).read()
    split=re.split(r"\n|\\n", txt)
    string=split[8]
    if 'true' in string:
        shutil.copyfile(file,'per_curiam')

KeyboardInterrupt: 

Not working at the moment but lets try pulling dates. the date can be found easily enough in the 'readable' version....

In [50]:
lit=op95_readable[10]
print(lit[-22:-12])

2010-03-13


...Lets see if i can make a loop that puts the dates in a list

In [9]:
dates=[]
ca10='/Users/keira.richards/E_DHML/embed/ca10/'
for file_name in os.listdir(ca10):
    file=os.path.join(ca10,file_name)
    txt=open(file).read()
    split=re.split(r"\n|\\n", txt)
    date_string=split[10]
    date_actual= date_string[-22:-12:]
    dates.append(date_actual)
len(dates)

KeyboardInterrupt: 

That kind of worked...when I run a random sample we can see it isnt perfectly pulling dates and there is one extra (???), but I'll come back to that later.


### Non-ML Analyis
Im going to try to do some term frequency analayis on a random sample of the Json files. After muplitplee attempts to use the whole set, I dont think my 2015 Mac has the RAM. But he tried and im proud of him. I primarily followed Melanie Walsh's [tutorial](https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html) with bits of code to troubleshoot, mostly from the Stackoverflow, googled as needed.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path 
import glob
import random
json_files = glob.glob('/Users/keira.richards/E_DHML/embed/ca10/*' + "*.json")

In [2]:
json_sample=random.sample (json_files, 1000)
titles = [Path(text).stem for text in json_sample]

In [3]:
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')
tfidf_vector = tfidf_vectorizer.fit_transform(json_sample)

In [4]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())

In [5]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()
tfidf_slice = tfidf_df[['government', 'men', 'man', 'women', 'woman', 'race','rights', 'children']]
tfidf_slice.sort_index().round(decimals=2)

,government,men,man,women,woman,race,rights,children
00_Document Frequency,368.00,57.0,83.0,27.0,26.00,46.0,409.00,69.0
1033267,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
1034289,0.03,0.0,0.0,0.0,0.00,0.0,0.00,0.0
1035771,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
1038538,0.00,0.0,0.0,0.0,0.01,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...
859174,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0
866939,0.00,0.0,0.0,0.0,0.00,0.0,0.01,0.0
899668,0.00,0.0,0.0,0.0,0.00,0.0,0.01,0.0
986,0.00,0.0,0.0,0.0,0.00,0.0,0.01,0.0


In [6]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,4547829,00,0.005761
1,4547829,000,0.000000
2,4547829,00002,0.000000
3,4547829,00004,0.000000
4,4547829,00007,0.000000
...,...,...,...
64508995,155695,zwickler,0.000000
64508996,155695,zwygart,0.000000
64508997,155695,zygoma,0.000000
64508998,155695,ætna,0.000000


In [7]:
tfidf_df = tfidf_df.stack().reset_index()
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

In [8]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

#### TF-IDF analysis: 
Even using just about 2% of total cases, the TF-IDF heatmap is interesting. Throughout the visualization, the highest scored terms appear to be names that are related to the specific case. For example, 1035771 must have involved an entity called "beauclair", as it has an .8 tf-idf score. Another case has high tf-idf scores for "halliburton", "sinclair" and "negligence" signaling the case is probably prosecuting the oil industry. I find this visualization is great for getting an idea for the case matter. One intriguingly frequent term that stood out to me was "class". it was only after searching a couple cases myself I found this was an aspect of how the Json files were formatted an represented nothing in terms of the case itself. This is a great learning opportunity in how the layout of my corpus can have a huge impact on the results of analysis.

In [12]:
import altair as alt
import numpy as np

#disable max rows error
alt.data_transformers.disable_max_rows()

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)


# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)


alt.LayerChart(...)